In [1]:
import os 
import folium   
from folium import plugins
import pandas as pd
import numpy as np
import datetime
    

# ## Data Prep

In [2]:
process=False
#process=True

if process:    
    data=pd.read_csv("export_european_capitals.csv")
    data.head()
    data.sort_values("ID_event").head()

    print(len(data))

    for line in data.itertuples():
        if data.iloc[line.Index]['dist']==0:
            data['dist'].at[line.Index]=7 #arbitrary for milan
    data.head()   

    data.sort_values(["ID_event","dist"], inplace=True)
    data

    ddrop=data.drop_duplicates(subset="ID_event" , keep="last")
    ddrop=ddrop.drop("ID_member", axis=1)
    #ddrop.sort_values("ID_event")
    ddrop=ddrop.reset_index()
    ddrop.head()

    d=data.groupby("ID_event")['dist'].quantile(0.75).reset_index()
    print(d.head())
    d.rename(columns={'dist':'maxdist'}, inplace=True)
    d2=data.groupby("ID_event")['dist'].mean().reset_index()
    d2.rename(columns={'dist':'meandist'}, inplace=True)
    ddrop=ddrop.merge(d, on="ID_event")
    ddrop=ddrop.merge(d2, on="ID_event")
    d3=data.groupby("ID_event")['dist'].median().reset_index()
    d3.rename(columns={'dist':'mediandist'}, inplace=True)
    ddrop=ddrop.merge(d3, on="ID_event")
    
    print(ddrop.head())

    d1=data.groupby("ID_event")['ID_member'].count().reset_index()
    print(d1.head())

    d1.rename(columns={'ID_member':'count_part'}, inplace=True)
    ddrop=ddrop.merge(d1, on="ID_event")
    del d1
    del d2
    del d
    del d3
    del data
    ddrop.head()
    event_df=pd.read_csv("../../csv/struttura/event.csv")

    event_df['date']=str
    for line in event_df.itertuples():
        try:
            event_df['date'].at[line.Index]=str(datetime.datetime.fromtimestamp(int(float(line.time)/1000)).strftime('%c'))
        except:
            event_df['date'].at[line.Index]="Non disponibile"
    #event_df.head() 
    
    ddrop=ddrop.merge(event_df, left_on="ID_event", right_on="_key")
    #ddrop.head()
    del event_df
    topic_df=pd.read_csv("event_topic_keywords.csv")
    ddrop=ddrop.merge(topic_df,left_on="ID_event", right_on="ID").reset_index().drop("level_0",1)
    del topic_df
    ddrop.to_csv("post_proc.csv")
    
    

else:
    ddrop=pd.read_csv("post_proc.csv")
#ddrop1=pd.read_csv("post_proc.csv")
ddrop.head()
#topic_df.head()

,Unnamed: 0,index,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist,...,count_part,_key,event_name,event_url,time,date,ID,City,Country,Keywords
0,0,157797,106866582,Paris,48.86,2.34,NaN,NaN,NaN,7.000000,...,1,106866582,Envie d\u00eatre form\u00e9 \u00e0 la M\u00e9d...,https://www.meetup.com/Lets-talk-about-relatio...,NONE,Non disponibile,106866582,Paris,fr,"['Support', 'Social', 'Education', 'Well-being..."
1,1,161868,115197702,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,7.000000,...,1,115197702,Carlinada en Madrid - Pug Playtime in Madrid,https://www.meetup.com/MadridPugClub/events/11...,NONE,Non disponibile,115197702,Madrid,es,['Pets']
2,2,111343,123953142,London,51.55,-0.17,NaN,NaN,NaN,5.886219,...,1,123953142,Game Development Workshop Series - Session 05,https://www.meetup.com/London-Game-Developers/...,NONE,Non disponibile,123953142,London,gb,['Tech']
3,3,23139,128105672,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,7.000000,...,1,128105672,Dinner and Drinks Cruise @ IJ Amsterdam,https://www.meetup.com/Americans-in-Amsterdam/...,NONE,Non disponibile,128105672,Amsterdam,nl,"['Social', 'Singles', 'Food & Drink', 'Languag..."
4,4,122132,131523832,London,51.50,-0.19,NaN,NaN,NaN,7.939657,...,1,131523832,Work in Fashion? Love Fashion?,https://www.meetup.com/Shop-Emerging-Fashion-D...,NONE,Non disponibile,131523832,London,gb,"['Social', 'Fashion', 'Photography', 'Business']"


In [3]:
for line in ddrop.itertuples():
    if line.Country=="it":
        #print(line.g_city)
        if line.g_city=="Milano":
            ddrop['Country'].at[line.Index]="it_Milano"
        if line.g_city=="Roma":
            ddrop['Country'].at[line.Index]="it_Roma"
            
ddrop[ddrop['Country']=="it_Milano"]

,Unnamed: 0,index,ID_event,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist,...,count_part,_key,event_name,event_url,time,date,ID,City,Country,Keywords
244,244,83647,255828197,Milano,45.46,9.19,Coworking SLAM,45.459090,9.172800,7.000000,...,2,255828197,Lezione #15 - Gestire lemotivit\u00e0 e gestir...,https://www.meetup.com/A-scuola-di-dibattito-M...,1562173200000,Wed Jul 3 19:00:00 2019,255828197,Milano,it_Milano,"['Well-being', 'Business', 'Education']"
565,565,161887,257680863,Milano,45.46,9.19,milan,45.465424,9.185925,10279.052175,...,4,257680863,La verit\u00e0 sulleCommerce,https://www.meetup.com/Milano-Retail-ed-eComme...,1557330600000,Wed May 8 17:50:00 2019,257680863,Milano,it_Milano,"['Business', 'Tech']"
811,811,14287,258124335,Milano,45.46,9.19,NaN,NaN,NaN,13.191883,...,4,258124335,Mini workshop di yoga della risata e yoga facc...,https://www.meetup.com/Yoga-della-Risata-C-e-t...,1554555600000,Sat Apr 6 15:00:00 2019,258124335,Milano,it_Milano,"['Well-being', 'Fitness']"
1358,1358,45740,258842222,Milano,45.46,9.19,NaN,NaN,NaN,4.078823,...,1,258842222,Passione Valtellina - Le declinazioni della Ch...,https://www.meetup.com/GoDrink/events/258842222/,1555612200000,Thu Apr 18 20:30:00 2019,258842222,Milano,it_Milano,"['Social', 'Food & Drink', 'Singles']"
1384,1384,51541,258868705,Milano,45.46,9.19,Microsoft House,45.481792,9.182817,843.046509,...,27,258868705,Build a Visually-Aware Recommender System \u20...,https://www.meetup.com/Milano-Data-Science-ODS...,1554397200000,Thu Apr 4 19:00:00 2019,258868705,Milano,it_Milano,"['Tech', 'Business']"
1433,1433,111303,258902824,Milano,45.46,9.19,Sede Microsoft,45.481972,9.182885,7.000000,...,2,258902824,Follow Your Sensation... a Microsoft story,https://www.meetup.com/agilelandscape/events/2...,1554309000000,Wed Apr 3 18:30:00 2019,258902824,Milano,it_Milano,"['Business', 'Tech']"
1740,1740,99470,259124091,Milano,45.46,9.19,Microsoft House,45.481792,9.182817,9608.126714,...,18,259124091,Introduzione alle GAN (Generative Adversarial ...,https://www.meetup.com/Milano-Data-Science-ODS...,1558544400000,Wed May 22 19:00:00 2019,259124091,Milano,it_Milano,"['Tech', 'Business']"
1782,1782,116563,259155721,Milano,45.46,9.19,Tenuta la Pila,45.138120,11.370935,155.494714,...,1,259155721,BGS GAME WEEKEND IN AGRITURISMO,https://www.meetup.com/BGS-Game-Night/events/2...,1556179200000,Thu Apr 25 10:00:00 2019,259155721,Milano,it_Milano,"['Games', 'Sci fi']"
1821,1821,102371,259186417,Milano,45.46,9.19,Vodafone Village,45.445740,9.123113,7.000000,...,7,259186417,Follow Your Sensation... a Microsoft story.,https://www.meetup.com/Vodafone-CatchUp/events...,1554224400000,Tue Apr 2 19:00:00 2019,259186417,Milano,it_Milano,"['Tech', 'Community', 'Business']"
2290,2290,49954,259440682,Milano,45.46,9.19,Porto di Cecina,43.304120,10.491035,476.564455,...,2,259440682,PASQUA a VELA in TOSCANA,https://www.meetup.com/il-mare-visto-dal-mare/...,1555693200000,Fri Apr 19 19:00:00 2019,259440682,Milano,it_Milano,"['Social', 'Outdoors', 'Photography', 'Arts', ..."


## Let's build a map

In [4]:
country_list=list(ddrop.Country.unique())
print(len(country_list))
d=ddrop.groupby("Country")['ID_event'].count()
rm1_list=[]
for elem in country_list:
    if d[elem] < 50:
        rm1_list.append(elem)

manual_rm_list=["ca", "ge", "ru", "cy", "us", "bg", "tr", "by", "am", "rs", "sk","si",  "li", "mk", "md", "ba", "gi", "ad"]#out of UE borders, or just too few events in there
rm_list=list(set(rm1_list+manual_rm_list))
#print(tot_list)
for elem in rm_list:
    country_list.remove(elem)
country_list

46


['fr',
 'es',
 'gb',
 'nl',
 'se',
 'de',
 'cz',
 'pl',
 'ie',
 'be',
 'lu',
 'no',
 'hu',
 'dk',
 'ch',
 'it_Milano',
 'at',
 'fi',
 'pt',
 'ro',
 'gr',
 'it_Roma',
 'ua',
 'hr']

In [9]:
pop={
    'fr':2141000,
    'es':3174000,
    'gb':8136000,
    'nl':821752,
    'se':965232,
    'de':3575000,
    'cz':1281000,
    'pl':1765000,
    'ie':544107,
    'be':174383,
    'lu':114303,
    'no':634293,
    'hu':1756000,
    'dk':602481,
    'ch':133115,
    'it_Milano':1352000,
    'at':1868000,
    'fi':631695,
    'pt':504718,
    'ro':1836000,
    'gr':3074160,
    'it_Roma':2865000,
    'ua':2884000,
    'hr':792875
}

In [10]:
ddrop=ddrop.loc[ddrop['Country'].isin(country_list)]

In [11]:
count_df=ddrop.groupby('Country', as_index=False).count()  
count_df=count_df[['Country', 'index']].reset_index()
count_df['norm_count']=0

In [12]:
for line in count_df.itertuples():
    norm_count=(line.index/pop[line.Country])*100000
    #norm_count
    count_df.norm_count.at[line.Index]=norm_count
count_df

,level_0,Country,index,norm_count
0,0,at,590,31
1,1,be,611,350
2,2,ch,126,94
3,3,cz,326,25
4,4,de,2476,69
5,5,dk,357,59
6,6,es,2232,70
7,7,fi,178,28
8,8,fr,2854,133
9,9,gb,9841,120


In [ ]:
capitals={
    'fr':"Paris",
    'es':"Madrid",
    'gb':"London",
    'nl':"Amsterdam",
    'se':"Stockholm",
    'de':"Berlin",
    'cz':"Praga",
    'pl':"Varsaw",
    'ie':"Dublin",
    'be':"Bruxelles",
    'lu':"Luxemburg",
    'no':"Oslo",
    'hu':"Budapest",
    'dk':"Copenaghen",
    'ch':"Berna",
    'it':"Rome",
    'at':"Vienna",
    'fi':"Helsinki",
    'pt':"Porto",
    'ro':"Bucarest",
    'gr':"Athens",
    'ua':"Kiev",
    'hr':"Zagabria",
}

In [37]:
from ipywidgets import interact

go=True
if go:
    topic_list=[]
    for line in ddrop.itertuples():
        #keywords=line.Keywords.split(",")
        #print(keywords)
            
        for topic in line.Keywords.split(","):
            Topic=[]
            topic=topic.replace("[","").replace("]","").replace("'","").replace(",","").replace(" ","")
            Topic.append(topic)
        for topic in Topic:
            if  topic not in topic_list:
                topic_list.append(topic)
count_dict={}
print(topic_list)
for elem in topic_list:
    if elem == "":
        topic_list.remove(elem)
print(topic_list)

['Business', 'Pets', 'Tech', 'Dancing', 'Fitness', 'Languages', 'BookClubs', 'Spirituality', 'Sports', 'Music', 'Education', 'Crafts', 'Singles', 'Movements', 'Food&Drink', 'Social', '', 'Beliefs', 'Outdoors', 'LGBT', 'Arts', 'Well-being', 'Games', 'Films', 'Support', 'Photography', 'Community', 'Scifi', 'Lifestyle', 'Fashion', 'Moms&Dads', 'Auto', 'Paranormal']


['Business',
 'Pets',
 'Tech',
 'Dancing',
 'Fitness',
 'Languages',
 'BookClubs',
 'Spirituality',
 'Sports',
 'Music',
 'Education',
 'Crafts',
 'Singles',
 'Movements',
 'Food&Drink',
 'Social',
 'Beliefs',
 'Outdoors',
 'LGBT',
 'Arts',
 'Well-being',
 'Games',
 'Films',
 'Support',
 'Photography',
 'Community',
 'Scifi',
 'Lifestyle',
 'Fashion',
 'Moms&Dads',
 'Auto',
 'Paranormal']

In [40]:
for country in count_df.Country:
    #print("#####\n",country, "\n#####\n")
    punt_count_dict={}
    for topic in topic_list:
        #print(topic)
        count_topic=0
        for line in ddrop.itertuples():
                if line.Country==country:    
                    keywords=line.Keywords
                    #print(keywords)
                    for word in keywords.split():
                        topic_curr=word.replace("[","").replace("]","").replace("'","").replace(",","").replace(" ","")
                        #Topic.append(topics)
                        if topic_curr==topic:
                            count_topic+=1
                            #print(count_topic)
        #print(topic,count_topic)
        punt_count_dict[topic]=count_topic
    #print(punt_count_dict)
    count_dict[country]=punt_count_dict

                #print (topic)
                #print (type(line.v_lat))
                #heat_data.append([float(line.v_lat),float(line.v_lon)])
#print(topic_list)
count_dict

{'at': {'Business': 116,
  'Pets': 0,
  'Tech': 159,
  'Dancing': 22,
  'Fitness': 119,
  'Languages': 106,
  'BookClubs': 0,
  'Spirituality': 85,
  'Sports': 82,
  'Music': 133,
  'Education': 61,
  'Crafts': 11,
  'Singles': 17,
  'Movements': 1,
  'Food&Drink': 0,
  'Social': 257,
  'Beliefs': 15,
  'Outdoors': 163,
  'LGBT': 1,
  'Arts': 119,
  'Well-being': 141,
  'Games': 62,
  'Films': 96,
  'Support': 13,
  'Photography': 6,
  'Community': 23,
  'Scifi': 0,
  'Lifestyle': 0,
  'Fashion': 0,
  'Moms&Dads': 0,
  'Auto': 0,
  'Paranormal': 0},
 'be': {'Business': 134,
  'Pets': 4,
  'Tech': 110,
  'Dancing': 60,
  'Fitness': 65,
  'Languages': 150,
  'BookClubs': 0,
  'Spirituality': 114,
  'Sports': 100,
  'Music': 39,
  'Education': 75,
  'Crafts': 10,
  'Singles': 44,
  'Movements': 17,
  'Food&Drink': 0,
  'Social': 193,
  'Beliefs': 6,
  'Outdoors': 52,
  'LGBT': 3,
  'Arts': 86,
  'Well-being': 136,
  'Games': 41,
  'Films': 21,
  'Support': 8,
  'Photography': 4,
  'Commun

In [20]:
@interact(topic=topic_list)
def show_canned_examples(topic):
    lat_df=ddrop[['v_lat', 'v_lon']]
    print(len(ddrop))
    str(lat_df.v_lat[0]) == "nan"
    
    for line in lat_df.itertuples():
        if str(line.v_lat) == "nan":
            lat_df['v_lat'].at[line.Index]= ddrop['g_lat'].at[line.Index]
        if str(line.v_lon) == "nan":
            lat_df['v_lon'].at[line.Index]= ddrop['g_lon'].at[line.Index]
    len(lat_df)

    lat_df.isna().any()

    lat_df=lat_df.astype(float)

    from folium.plugins import HeatMap


    map_hooray = folium.Map(location=[ddrop['v_lat'].mean(),ddrop['v_lon'].mean()], zoom_start=5)


    # List comprehension to make out list of lists
    #heat_data = [[row['v_lat'],row['v_lon']] for index, row in lat_df.iterrows()]
    heat_data=[]
    for line in lat_df.itertuples():
        if ddrop.Country.at[line.Index] in country_list:    
            keywords=ddrop.Keywords.at[line.Index]
            #print(keywords)
            for topics in keywords.split():
                topic_curr=topic.replace("[","").replace("]","").replace("'","").replace(",","").replace(" ","")
                #Topic.append(topics)
            if topic_curr==topic:
                
                #print (topic)
                #print (type(line.v_lat))
                #heat_data.append([float(line.v_lat),float(line.v_lon)])
    # Plot it on the map
    HeatMap(heat_data).add_to(map_hooray)

    # Display the map
    map_hooray.save("/home/dario/NeoMeetup/Visualizations/map/heat_try.html")
    display(map_hooray)

interactive(children=(Dropdown(description='topic', options=('Business', 'Pets', 'Tech', 'Dancing', 'Fitness',…

In [ ]:
from ipywidgets import interact

go=True
if go:
    topic_list=[]
    for line in ddrop.itertuples():
        #keywords=line.Keywords.split(",")
        #print(keywords)
            
        for topic in line.Keywords.split(","):
            Topic=[]
            topic=topic.replace("[","").replace("]","").replace("'","").replace(",","").replace(" ","")
            Topic.append(topic)
        for topic in Topic:
            if  topic not in topic_list:
                topic_list.append(topic)
        
    
#print(topic_list)
@interact(topic=topic_list)
def show_canned_examples(topic):

    lat_df=ddrop[['v_lat', 'v_lon']]
    print(len(ddrop))
    str(lat_df.v_lat[0]) == "nan"
    
    for line in lat_df.itertuples():
        if str(line.v_lat) == "nan":
            lat_df['v_lat'].at[line.Index]= ddrop['g_lat'].at[line.Index]
        if str(line.v_lon) == "nan":
            lat_df['v_lon'].at[line.Index]= ddrop['g_lon'].at[line.Index]
    len(lat_df)

    lat_df.isna().any()

    lat_df=lat_df.astype(float)

    from folium.plugins import HeatMap


    map_hooray = folium.Map(location=[ddrop['v_lat'].mean(),ddrop['v_lon'].mean()], zoom_start=4)


    # List comprehension to make out list of lists
    #heat_data = [[row['v_lat'],row['v_lon']] for index, row in lat_df.iterrows()]
    heat_data=[]
    #for row in count_df.itertuples():
    #    norm_count=0
        for line in lat_df.itertuples():
            if ddrop.Country.at[line.Index] == row.Country:
     #           norm_count+=1
                keywords=ddrop.Keywords.at[line.Index]
                #print(keywords)
                for topics in keywords.split():
                    topic=topic.replace("[","").replace("]","").replace("'","").replace(",","").replace(" ","")
                #Topic.append(topics)
                if topic in topic_list:
                    #print (topic)
                    #print (type(line.v_lat))
                    heat_data.append([float(line.v_lat),float(line.v_lon)])
                if norm_count==row.norm_count:
                    print("for country {} inserted {} events".format(row.Country, norm_count))
    # Plot it on the map
    HeatMap(heat_data).add_to(map_hooray)
    plugins.Fullscreen(

            position='bottomright',
            title='Expand me',
            title_cancel='Exit me',
            force_separate_button=True
        ).add_to(map_hooray)

    # Display the map
    map_hooray.save("/home/dario/NeoMeetup/Visualizations/map/heat_try.html")
    heat_dict[]

